# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication. **Please note that the "metadata" section must be filled as shown, including the ResourceId that must contain the connection name reporten on line 1**:
```
name: mmai-hub04-fvye-connection-AISearch
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net/
is_shared: true
metadata:
  ApiType: Azure
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-fvye-connection-AISearch
  type: azure_ai_search
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`


# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv(dotenv_path='../infra/credentials.env', override=True)
 
model_name = os.environ["MODEL_DEPLOYMENT_NAME"]
project_connection_string = os.environ["AI_PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...ing-rg;aifoundry-upskilling-pj>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType# <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

ImportError: cannot import name 'AzureAISearchQueryType' from 'azure.ai.projects.models' (/anaconda/envs/nico_env/lib/python3.10/site-packages/azure/ai/projects/models/__init__.py)

# List existing connections and highlight the "AI Search" ones

In [3]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

NameError: name 'project_client' is not defined

# Chosen AI Search Connection

In [4]:
conn_id

NameError: name 'conn_id' is not defined

# Initialize `AzureAISearchTool`, adding the connection id to it

In [5]:
ai_search = AzureAISearchTool(index_connection_id=conn_id, 
                              index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
                              # Comment next line if using a Portal Index with integrated vectorization
                              query_type=AzureAISearchQueryType.SIMPLE) #SEMANTIC, Dy default VECTOR SEMANTIC HYBRID
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

NameError: name 'conn_id' is not defined

# Create AI Foundry Agent

In [6]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="aisearch-agent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

NameError: name 'project_client' is not defined

# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'object': 'thread', 'created_at': 1744364239, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_hxe2qdavcybMgMbjTpHkUEbI', 'object': 'thread.message', 'created_at': 1744364240, 'assistant_id': None, 'thread_id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent synchronously

In [8]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: completed.

Run: {'id': 'run_Ye1t8onnNjZHql5vqcx9YZ7i', 'object': 'thread.run', 'created_at': 1744364241, 'assistant_id': 'asst_YNjFImN8HxuH1kn3DgA9ADVW', 'thread_id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'status': 'completed', 'started_at': 1744364242, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1744364244, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 3244, 'completion_tokens': 99, 'total_tokens': 3343, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: user 15.2 ms, sys: 2.99 ms, total: 18.2 ms


# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: The document does not explicitly mention the country with the lowest GDP in 2023, but it provides insights into real GDP deviations from pre-crisis trends across several regions. The United Kingdom exhibits the largest negative deviation, close to -4.0%, among the regions listed, which includes Japan, Canada, the Euro Area, the UK, and G-20 Emerging Markets【3:0†source】.
>>> Annotation in MessageTextContent 1 of message 2: 【3:0†source】



In [10]:
from azure.ai.projects.models import FilePurpose, MessageRole

# Get the last message from the sender
last_msg = messages.get_last_text_message_by_role(MessageRole.AGENT)
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: The document does not explicitly mention the country with the lowest GDP in 2023, but it provides insights into real GDP deviations from pre-crisis trends across several regions. The United Kingdom exhibits the largest negative deviation, close to -4.0%, among the regions listed, which includes Japan, Canada, the Euro Area, the UK, and G-20 Emerging Markets【3:0†source】.


# Print annotations from the messages

In [11]:
print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

for annotation in last_msg.text.annotations:
    print(annotation["text"], annotation["url_citation"]["url"])

Number of annotation(s): 1
【3:0†source】 https://ai-search-abutneva687267079310.search.windows.net/indexes/my-index-2025-04-10-2025-04-10-sdk/docs/2?api-version=2024-07-01&$select=id,content,title


# Run Steps

In [12]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_1n8FpoSRYQMefvycyZ9vRxHN', 'object': 'thread.run.step', 'created_at': 1744364243, 'run_id': 'run_Ye1t8onnNjZHql5vqcx9YZ7i', 'assistant_id': 'asst_YNjFImN8HxuH1kn3DgA9ADVW', 'thread_id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1744364244, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_6TTLVjVMYy49InnW4RDur88H'}}, 'usage': {'prompt_tokens': 2905, 'completion_tokens': 84, 'total_tokens': 2989, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_jBVJvXqTKbXOhKuI7IHGH7fO', 'object': 'thread.run.step', 'created_at': 1744364242, 'run_id': 'run_Ye1t8onnNjZHql5vqcx9YZ7i', 'assistant_id': 'asst_YNjFImN8HxuH1kn3DgA9ADVW', 'thread_id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 

# START Teardown

In [13]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'object': 'thread', 'created_at': 1744364239, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_AceRW6neQgjv3ow8mvYM2Mb8', 'object': 'thread.deleted', 'deleted': True}

In [14]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

11 agent(s) will now be deleted

1 - Agent aisearch-agent has been deleted

2 - Agent aisearch-agent has been deleted

3 - Agent aisearch-agent has been deleted

4 - Agent aisearch-agent has been deleted

5 - Agent aisearch-agent has been deleted

6 - Agent aisearch-agent has been deleted

7 - Agent aisearch-agent has been deleted

8 - Agent aisearch-agent has been deleted

9 - Agent aisearch-agent has been deleted

10 - Agent aisearch-agent has been deleted

11 - Agent Agent318 has been deleted


# HIC SUNT LEONES